In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [10]:
from fastai.vision import *
from fastai import * 
from fastai.text import *
from tqdm import tqdm
tqdm.pandas(desc="my bar!")

In [11]:
pip install torchtext

Note: you may need to restart the kernel to use updated packages.


In [12]:
import torchtext
from torchtext.data import Iterator, BucketIterator
from torchvision import transforms, utils


In [13]:
path = Config.data_path()
path.mkdir(parents=True, exist_ok=True)

path.ls()

In [14]:
df_train = pd.read_csv(path/'train.csv')
df_test = pd.read_csv(path/'test.csv')
# df_train

In [15]:
import string
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]



mispell_dict = {"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"}

def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

def mispelled_words(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispellings, mispellings_re = mispelled_words(mispell_dict)

def replace_typical_mispell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace,text)

def clean_text(x):

    x = str(x)
    for punct in puncts:
        x.translate(str.maketrans('', '', string.punctuation))
    return x



In [16]:
df_train["question_text"] = df_train["question_text"].str.lower()
df_test["question_text"] = df_test["question_text"].str.lower()

df_train['question_text'] = df_train['question_text'].progress_apply(lambda x: clean_text(x))
df_test['question_text'] = df_test['question_text'].progress_apply(lambda x: clean_text(x))

df_train['question_text'] = df_train['question_text'].progress_apply(lambda x: clean_numbers(x))
df_test['question_text'] = df_test['question_text'].progress_apply(lambda x:clean_numbers(x))

df_train['question_text'] = df_train['question_text'].progress_apply(lambda x: replace_typical_mispell(x))
df_test['question_text'] = df_test['question_text'].progress_apply(lambda x: replace_typical_mispell(x))

train_x = df_train['question_text'].fillna('_##_').values
test_x = df_test['question_text'].fillna('_##_').values

my bar!: 100%|██████████| 375806/375806 [00:05<00:00, 71190.36it/s]


import zipfile
zf = zipfile.ZipFile(path/'test.csv.zip')
df_test = pd.read_csv(zf.open('test.csv'))
zf = zipfile.ZipFile(path/'train.csv.zip')
df_train = pd.read_csv(zf.open('train.csv'))



In [14]:
print(df_train.head(3))

                    qid                                      question_text  \
0  00002165364db923c7e6  how did quebec nationalists see their province...   
1  000032939017120e6e44  do you have an adopted dog, how would you enco...   
2  0000412ca6e4628ce2cf  why does velocity affect time? does velocity a...   

   target  
0       0  
1       0  
2       0  


In [49]:
print(df_train.shape)
print(df_test.shape)

(1306122, 3)
(375806, 2)


In [50]:
 df_train.head(3)

,qid,question_text,target
0,00002165364db923c7e6,how did quebec nationalists see their province...,0
1,000032939017120e6e44,"do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,why does velocity affect time? does velocity a...,0


In [5]:
def prepare_sequences():
    tokenizer = lambda text: text.split() # the function above is the function we will be using to tokenize the text
    TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer, lower=True)
    LABEL = torchtext.data.Field(sequential=False, use_vocab=False) # sequential and use_vocab=False since no text (binary)
    train_datafields = [("qid", None), ("question_text", TEXT), ("target", LABEL)]
    train = torchtext.data.TabularDataset( # If we had a validation set as well, we would add an additional .splits(...)
                        # path="data/train_cleaned_v2.csv", # the root directory where the data lies
                        path = "/Users/caitlinaspinall/.fastai/data/train.csv",
                        format='csv',
                        # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
                        skip_header=True, 
                        fields=train_datafields)
    test_datafields = [("qid", None),
                     ("question_text", TEXT), ("target", LABEL)] 
    test = torchtext.data.TabularDataset( 
                path="/Users/caitlinaspinall/.fastai/data/test.csv",
                format="csv",
                skip_header=True,
                fields=test_datafields)
    return TEXT, LABEL, train, test
# Chat Conversation End
# Type a message...



In [5]:
TEXT,LABEL,train,test = prepare_sequences()


In [17]:
print(df_train)


                          qid  \
0        00002165364db923c7e6   
1        000032939017120e6e44   
2        0000412ca6e4628ce2cf   
3        000042bf85aa498cd78e   
4        0000455dfa3e01eae3af   
5        00004f9a462a357c33be   
6        00005059a06ee19e11ad   
7        0000559f875832745e2e   
8        00005bd3426b2d0c8305   
9        00006e6928c5df60eacb   
10       000075f67dd595c3deb5   
11       000076f3b42776c692de   
12       000089792b3fc8026741   
13       000092a90bcfbfe8cd88   
14       000095680e41a9a6f6e3   
15       0000a89942e3143e333a   
16       0000b8e1279eaa0a7062   
17       0000bc0f62500f55959f   
18       0000ce6c31f14d3e09ec   
19       0000d329332845b8a7fa   
20       0000dd973dfd35508c16   
21       0000e4d455f9c8877dc9   
22       0000e91571b60c2fb487   
23       000101ac65db6e4a1c13   
24       00010632971fe5e3e0e2   
25       00010a2e064c3e8f152a   
26       00012011b6c7759461e8   
27       00012fd5128d576260ab   
28       0001303b1799a042b26b   
29       0

In [25]:

from torchtext.data import Iterator, BucketIterator
train_iter = BucketIterator(
    train, # we pass in the datasets we want the iterator to draw data from
    batch_size=64, 
    sort_key= lambda x: len(x.question_text), # the BucketIterator needs to be told what function it should use to group the data.
    sort_within_batch=False, # sorting would add bias
    repeat=False)
test_iter = Iterator(test, batch_size = 64, device = None, sort = False,  sort_within_batch = False, repeat = False) #Changed device from -1 to None


In [26]:
iter(train_iter).__next__().question_text

TypeError: object of type 'module' has no len()